In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
import datetime
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
from sklearn.linear_model import RidgeClassifier

In [2]:
# remove commas from weather data
new_file = ''
with open('../data/weather.csv') as f:
    with open('../data/new.csv', 'a') as nf:
        for line in f:
            elements = line.split(',')
            print(elements)
            print(len(elements))
            break
            for i, element in enumerate(elements):
                new_element = ''
                for char in element:
                    if char != ',' and char !='"' and char != '"' and char != ' ':
                        new_element += char
                
                if i != len(element)-1:
                    nf.write(f'{new_element},') 
                else:
                    nf.write(f'{new_element}\n')

['"STATION"', '"NAME"', '"LATITUDE"', '"LONGITUDE"', '"ELEVATION"', '"DATE"', '"AWND"', '"AWND_ATTRIBUTES"', '"FMTM"', '"FMTM_ATTRIBUTES"', '"PGTM"', '"PGTM_ATTRIBUTES"', '"PRCP"', '"PRCP_ATTRIBUTES"', '"SNOW"', '"SNOW_ATTRIBUTES"', '"SNWD"', '"SNWD_ATTRIBUTES"', '"TAVG"', '"TAVG_ATTRIBUTES"', '"TMAX"', '"TMAX_ATTRIBUTES"', '"TMIN"', '"TMIN_ATTRIBUTES"', '"WDF2"', '"WDF2_ATTRIBUTES"', '"WDF5"', '"WDF5_ATTRIBUTES"', '"WESD"', '"WESD_ATTRIBUTES"', '"WSF2"', '"WSF2_ATTRIBUTES"', '"WSF5"', '"WSF5_ATTRIBUTES"', '"WT01"', '"WT01_ATTRIBUTES"', '"WT02"', '"WT02_ATTRIBUTES"', '"WT03"', '"WT03_ATTRIBUTES"', '"WT05"', '"WT05_ATTRIBUTES"', '"WT07"', '"WT07_ATTRIBUTES"', '"WT08"', '"WT08_ATTRIBUTES"', '"WT09"', '"WT09_ATTRIBUTES"', '"WT13"', '"WT13_ATTRIBUTES"', '"WT14"', '"WT14_ATTRIBUTES"', '"WT16"', '"WT16_ATTRIBUTES"', '"WT21"', '"WT21_ATTRIBUTES"\n']
56


In [3]:
weather_data = pd.read_csv('../data/weather.csv')

col_to_nan = {}
for col in weather_data.columns:
    percent_nan = weather_data[col].isna().sum() / 4017
    
    if percent_nan > .6:
        weather_data.drop(col, axis=1, inplace=True)

to_drop = ['NAME' , 'FMTM', 'FMTM_ATTRIBUTES', 'AWND', 'LONGITUDE', 'LATITUDE', 'ELEVATION', 'AWND_Attributes', 
           'SNOW_ATTRIBUTES', 'SNWD', 'PRCP_ATTRIBUTES', 'SNOW_ATTRIBUTES', 'TAVG_ATTRIBUTES', 
           "WDF2","WDF2_ATTRIBUTES","WDF5","WDF5_ATTRIBUTES", 'TAVG',
           "WESD_ATTRIBUTES", 'WESD', "WSF2","WSF2_ATTRIBUTES","WSF5","WSF5_ATTRIBUTES", 'STATION']


for col in weather_data.columns:
    if col in to_drop or 'ATTRIBUTES' in col:
        weather_data.drop(col, axis=1, inplace=True)

        
weather_data['DATE'] = weather_data['DATE'].apply(lambda x: f'{x[5:7]}/{x[8:]}/{x[:4]}')
weather_data

,DATE,PRCP,TMAX,TMIN
0,01/01/2009,0.0,54,45
1,01/02/2009,0.0,53,48
2,01/03/2009,0.0,60,50
3,01/04/2009,0.0,63,46
4,01/05/2009,0.0,58,42
...,...,...,...,...
4012,12/27/2019,0.0,62,43
4013,12/28/2019,0.0,63,46
4014,12/29/2019,0.0,62,47
4015,12/30/2019,0.0,64,53


In [4]:
flight_data = pd.read_csv('../data/Detailed_Statistics_Arrivals.csv', skiprows=7)
flight_data.drop(441999, inplace=True)
flight_data

flight_data.set_axis(['Carrier Code', 'DATE', 'Flight Number', 'Tail Number',
       'Origin Airport', 'Scheduled Arrival Time', 'Actual Arrival Time',
       'Scheduled Elapsed Time (Minutes)', 'Actual Elapsed Time (Minutes)',
       'Arrival Delay (Minutes)', 'Wheels-on Time', 'Taxi-In time (Minutes)',
       'Delay Carrier (Minutes)', 'Delay Weather (Minutes)',
       'Delay National Aviation System (Minutes)', 'Delay Security (Minutes)',
       'Delay Late Aircraft Arrival (Minutes)'], axis=1, inplace=True)

all_data = flight_data.merge(weather_data,on='DATE',how='left')

all_data['TAVG'] = (all_data['TMAX'] + all_data['TMIN']) / 2

all_data = all_data.drop(['TMAX', 'TMIN'], axis=1)
all_data

,Carrier Code,DATE,Flight Number,Tail Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Wheels-on Time,Taxi-In time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),PRCP,TAVG
0,WN,01/01/2009,12.0,N414WN,ABQ,12:45,13:00,125.0,112.0,15.0,12:54,6.0,1.0,0.0,0.0,0.0,14.0,0.0,49.5
1,WN,01/01/2009,21.0,N401WN,HOU,14:00,13:56,215.0,200.0,-4.0,13:50,6.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5
2,WN,01/01/2009,72.0,N399WN,OAK,11:20,11:14,80.0,68.0,-6.0,11:08,6.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5
3,WN,01/01/2009,75.0,N318SW,SMF,19:15,19:11,85.0,76.0,-4.0,19:03,8.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5
4,WN,01/01/2009,87.0,N430WN,MDW,14:05,13:53,275.0,255.0,-12.0,13:46,7.0,0.0,0.0,0.0,0.0,0.0,0.0,49.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
441994,WN,12/31/2019,6467.0,N734SA,OAK,07:30,07:15,90.0,76.0,-15.0,07:11,4.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0
441995,WN,12/31/2019,6516.0,N267WN,DEN,18:55,18:39,155.0,125.0,-16.0,18:31,8.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0
441996,WN,12/31/2019,6614.0,N8650F,MSY,20:25,20:17,265.0,251.0,-8.0,20:09,8.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0
441997,WN,12/31/2019,6667.0,N797MX,OAK,11:35,11:21,90.0,72.0,-14.0,11:13,8.0,0.0,0.0,0.0,0.0,0.0,0.0,62.0


In [5]:
weather_data

,DATE,PRCP,TMAX,TMIN
0,01/01/2009,0.0,54,45
1,01/02/2009,0.0,53,48
2,01/03/2009,0.0,60,50
3,01/04/2009,0.0,63,46
4,01/05/2009,0.0,58,42
...,...,...,...,...
4012,12/27/2019,0.0,62,43
4013,12/28/2019,0.0,63,46
4014,12/29/2019,0.0,62,47
4015,12/30/2019,0.0,64,53


In [6]:
delayed_flights = all_data['Arrival Delay (Minutes)'].copy()
for i, time in enumerate(delayed_flights):
    if time <= 15:
        delayed_flights[i] = 'Not Delayed'
    else:
        delayed_flights[i] = 'Delayed'
delayed_flights

0         Not Delayed
1         Not Delayed
2         Not Delayed
3         Not Delayed
4         Not Delayed
             ...     
441994    Not Delayed
441995    Not Delayed
441996    Not Delayed
441997    Not Delayed
441998    Not Delayed
Name: Arrival Delay (Minutes), Length: 441999, dtype: object

In [7]:
# counting number of flights in each class
num_on_time = 0
num_delayed = 0
num_early = 0
for delay_class in delayed_flights: # fix class imbalance - subset selection, etc.
    if delay_class == 'Not Delayed':
        num_on_time += 1
    elif delay_class == 'Early':
        num_early += 1
    else:
        num_delayed += 1

print('num_on_time:', num_on_time)
print('num_delayed:', num_delayed)
print('num_early:', num_early)

num_on_time: 343675
num_delayed: 98324
num_early: 0


In [8]:
# write function to convert date-time to month
num_to_month = {1: 'January', 2: 'February', 3: 'March', 4: 'April', 5: 'May', 6: 'June', 7: 'July', 8: 'August',
               9: 'September', 10: 'October', 11: 'November', 12: 'December'}
def time_to_minute(time):
    hours, minutes = time.split(':')
    return 60*int(hours) + int(minutes)

def extract_month(date):
    month = int(date.split('/')[0])
    if month > 12:
        print(f'month: {month}; date: {date}')
    return num_to_month[month]

In [9]:
# gather predictors
predictors = all_data.drop(['Carrier Code', 'Tail Number', 'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)', 
                            'Delay Weather (Minutes)', 'Delay Security (Minutes)', 
                            'Delay Late Aircraft Arrival (Minutes)', 'Actual Elapsed Time (Minutes)', 
                            'Actual Arrival Time', 'Flight Number', 'Wheels-on Time', 'Arrival Delay (Minutes)', 
                            'Delay National Aviation System (Minutes)'], 
                                axis = 1)


# change date to month (then get dummies), change time of day to minutes, get dummies for airports, make dummies for
# arrival status
predictors['Month'] = predictors['DATE'].apply(extract_month)
month_dummies = pd.get_dummies(predictors['Month'])
predictors['Scheduled Arrival Time'] = predictors['Scheduled Arrival Time'].apply(time_to_minute)
origin_dummies = pd.get_dummies(predictors['Origin Airport'])

predictors[list(month_dummies.columns)] = month_dummies
predictors[list(origin_dummies)] = origin_dummies

predictors.drop(['Origin Airport'], axis=1, inplace=True)
predictors['DATE'] = predictors['DATE'].apply(lambda x: x.replace('/', '-'))
predictors.set_index('DATE', inplace=True)
predictors.drop('Month', axis=1, inplace=True)
predictors

,Scheduled Arrival Time,Scheduled Elapsed Time (Minutes),PRCP,TAVG,April,August,December,February,January,July,...,PIT,RNO,SAT,SFO,SJC,SLC,SMF,STL,TPA,TUS
DATE,,,,,,,,,,,,,,,,,,,,,
01-01-2009,765,125.0,0.0,49.5,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
01-01-2009,840,215.0,0.0,49.5,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
01-01-2009,680,80.0,0.0,49.5,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
01-01-2009,1155,85.0,0.0,49.5,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
01-01-2009,845,275.0,0.0,49.5,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12-31-2019,450,90.0,0.0,62.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12-31-2019,1135,155.0,0.0,62.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12-31-2019,1225,265.0,0.0,62.0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
predictors.columns

Index(['Scheduled Arrival Time', 'Scheduled Elapsed Time (Minutes)', 'PRCP',
       'TAVG', 'April', 'August', 'December', 'February', 'January', 'July',
       'June', 'March', 'May', 'November', 'October', 'September', 'ABQ',
       'ATL', 'AUS', 'BHM', 'BNA', 'BOI', 'BWI', 'CMH', 'DAL', 'DEN', 'ELP',
       'HOU', 'IND', 'LAS', 'MCI', 'MDW', 'MKE', 'MSY', 'OAK', 'OKC', 'OMA',
       'PDX', 'PHX', 'PIT', 'RNO', 'SAT', 'SFO', 'SJC', 'SLC', 'SMF', 'STL',
       'TPA', 'TUS'],
      dtype='object')

In [11]:
status_to_num = {'Not Delayed': 0, 'Delayed': 1}
y = delayed_flights.apply(lambda x: status_to_num[x])
X = predictors.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
# need to figure out how many components work best
# encode output variables as numbers

# cross validate to find optimal number of components 
n_components = {'n_components': [i for i in range(1, 2)]}

lda_clf = LDA(shrinkage='auto', solver='eigen')
gscv_clf = GridSearchCV(lda_clf, param_grid=n_components)

gscv_clf.fit(X_train, y_train)
best_n = n_components['n_components'][gscv_clf.cv_results_['mean_test_score'].argmax()]
best_n

1

In [14]:
# refit with best n and find test score
y_tot = delayed_flights.apply(lambda x: status_to_num[x])
X_tot =  predictors.copy()
lda_clf = LDA(shrinkage='auto', solver='eigen')
lda_clf.fit(X_tot.copy(), y_tot.copy())
y_pred = lda_clf.predict(X_test)
mean_squared_error(y_test, y_pred)

0.22248868778280542

In [17]:
# sns.lineplot(x=n_neighbors['n_neighbors'], y=list(knn_cv.cv_results_['mean_test_score']))
# plt.show()

In [35]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss').fit(X_train, y_train)
y_pred = model.predict(X_test)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
accuracy = (tp + tn) / (tp+ tn+ fn+ fp)
print(f'Test Accuracy: {accuracy}')

/Users/ColeHoward/opt/anaconda3/lib/python3.9/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


Test Accuracy: 0.7875188536953243


In [37]:
pca = PCA(n_components = .95)

lin_combinations = pca.fit_transform(X_train)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss').fit(lin_combinations, y_train)
y_pred = model.predict(pca.transform(X_test))

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
accuracy = (tp + tn) / (tp+ tn+ fn+ fp)
print(f'accuracy: {accuracy}')


accuracy: 0.7784087481146305


In [38]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state=42, n_jobs=-1)
X_res, y_res = sm.fit_resample(X_train, y_train)

In [47]:
params = {"alpha": [.00001, .0001, .001, 1]}

cv_ridge = GridSearchCV(RidgeClassifier(), params).fit(X_res, y_res)

best_alpha = params['alpha'][np.argmax(cv_ridge.cv_results_['mean_test_score'])]
ridge = RidgeClassifier(alpha=best_alpha).fit(X_res, y_res)

y_pred = ridge.predict(X_test)

print(f'Test MSE: {mean_squared_error(y_test, y_pred)}')
print(f'Test Accuracy: {ridge.score(X_test, y_test)}')

Test MSE: 0.25871794871794873
Test Accuracy: 0.7412820512820513


In [45]:
# ridge Classifier without SMOTE
clf = RidgeClassifier().fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(f'Test MSE: {mean_squared_error(y_test, y_pred)}')
print(f'Test Accuracy: {clf.score(X_test, y_test)}')

Test MSE: 0.22139517345399698
Test Accuracy: 0.778604826546003


In [34]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(f'Test Accuracy: {clf.score(X_test, y_test)}')
print(f'Test MSE: {mean_squared_error(y_test, y_pred)}')

Test Accuracy: 0.7764253393665158
Test MSE: 0.22357466063348416


In [ ]:
import pandas as pd
import requests
from array import array

import requests
import math


# ## Loading historical weather data

# In[23]:





# # ## Loading current weather

# # In[14]:


# api_key = '511fe526899b1daec72be634b63cd99f'

# # lat and lon for LAX
# input_lat = 33.9416
# input_lon = -118.4085


# # In[15]:


# # convert to strings for api call
# lat = str(input_lat)
# lon = str(input_lon)


# # In[16]:


# api_url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + lon + \
#           '&exclude=hourly,daily&appid=' + api_key

# response = requests.get(api_url)
# current_weather = response.json()['current']
# current_weather


# # In[17]:


# curr_temp_k = current_weather['temp']

# # convert temp from kelvin to fahrenheit
# curr_temp = int(math.ceil((curr_temp_k - 273.15) * 9/5 + 32))
# curr_temp


# # In[ ]:





In [ ]:
# url = 'https://flight-friend-b2c01-default-rtdb.firebaseio.com/predictors.json'
# url = 'https://flight-friend-la-default-rtdb.firebaseio.com/predictors.json' 
url = 'https://flight-prediction-8e469-default-rtdb.firebaseio.com/predictors.json'

# get_url

# data = predictors.head()

# data_dict = data.to_dict()

response = requests.put(url, json=data_dict)
print(len(response.json()))

In [ ]:
data_dict = {}
predictors['status'] = list(delayed_flights)
for i, row in predictors.iterrows():
    data_dict[i] = row.to_dict()



In [ ]:
delayed_flights

In [ ]:
import json
import requests
url = 'https://flight-prediction-8e469-default-rtdb.firebaseio.com/predictors.json'
response = requests.get(url)
response_dict = response.json()

indexes = []
col_to_values = {col: [] for col in predictors.columns}
for index, col_to_val in response_dict.items():
    indexes.append(index)
    for col, val in col_to_val.items():
        col_to_values[col].append(val)
new_df = pd.DataFrame(col_to_values, index=indexes)
new_df

In [ ]:
len(predictors.columns)

In [ ]:
col_to_values.keys()

In [ ]:
predictors.columns

In [ ]:
api_key = '511fe526899b1daec72be634b63cd99f'

# lat and lon for LAX
input_lat = 33.9416
input_lon = -118.4085

lat = str(input_lat)
lon = str(input_lon)

api_url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + lon + \
          '&exclude=hourly,daily&appid=' + api_key

response = requests.get(api_url)
current_weather = response.json()['current']

curr_temp_k = current_weather['temp']

# convert temp from kelvin to fahrenheit
curr_temp = int(math.ceil((curr_temp_k - 273.15) * 9/5 + 32))


In [ ]:
import requests
api_key = '511fe526899b1daec72be634b63cd99f'

# lat and lon for LAX
input_lat = 33.9416
input_lon = -118.4085

lat = str(input_lat)
lon = str(input_lon)

api_url = 'https://api.openweathermap.org/data/2.5/onecall?lat=' + lat + '&lon=' + lon + \
            '&exclude=hourly,daily&appid=' + api_key

response = requests.get(api_url)
current_weather = response.json()['minutely']

max_precip = 0
for tup in current_weather:
    if tup['precipitation'] > max_precip:
        max_precip = tup['precipitation']

        print(max_precip)


In [ ]:
max_precip

In [ ]:
predictors

In [ ]:
new_df = pd.DataFrame(columns=predictors.columns)

In [ ]:
new_df.loc[0] = [0 for i in range(len(predictors.columns))]
new_df